## Part 1

In [41]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print("Imported necessary dependencies.")

Imported necessary dependencies.


In [19]:
#Load the URL and scrape the tables with html tag <table>
table=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
#the first table contains the informaiton I am need and it is at 0th index
df=table[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


The following code will drop rows with "Not assigned" value for Borough attribute/column.

In [34]:
df2=df.loc[(df['Borough'] != 'Not assigned')].reset_index(drop=True)
df2.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


To check if there are any postal codes with multiple neighborhoods in different columns.
Since it returned true when I run it, the problem doesn't exist.
I also assume that this data on Wikipedia page will not be modified anytime soon.

In [42]:
df2['Postal Code'].value_counts().max() == df2['Postal Code'].value_counts().min()

True

In [43]:
df2.shape

(103, 3)

There are 103 rows and 3 columns as expected.

## Part 2

I tried the geocoder function. It didn't work so I am going to load the CSV file with lat lng data.

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

print(latitude)

In [53]:
import io
import requests
url="https://cocl.us/Geospatial_data"
s=requests.get(url).content
latlng_data=pd.read_csv(io.StringIO(s.decode('utf-8')))
latlng_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
df3=pd.merge(df2,latlng_data,on='Postal Code')
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [55]:
df3.loc[df3['Postal Code']=='M5G']

,Postal Code,Borough,Neighborhood,Latitude,Longitude
24,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
